In [1]:
# Third party imports
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Local application imports
from smile.smile import Population, PopulationList
from smile.helper import truncatednormal
from smile.global_params import *

In [3]:
# Settings
seed = 3 # chosen by fair dice roll. guaranteed to be random. https://xkcd.com/221/
np.random.seed(seed)
np.set_printoptions(edgeitems=30, linewidth=100000)

## Experiment 0: Recreating the poster

Purpose:  
   * Recreate the data from the previous experiments summarized in the poster, to weed out coding errors or simulation/methodology discrepencies  
   
Hypothesis:  
   * Bias will be the same as the poster.  
   
Reasoning:  
   * Based on the same ideas  

In [6]:
# Simulation definition

def get_poster_populations(slope_option, error_option, npersons=100, npops=100):
    '''
    returns a PopulationList similar to the one described in the poster
    slope_option is 1, 2, or 3
    error_option is 30/100 or 50/100
    '''
    
    # Define and set visual score function
    pop = Population(npersons, f'poster with {slope_option} and {error_option}')
    gen_visualscores = lambda t,r,v0: np.maximum(-r*t+v0, VMIN)
    pop.set_score_generator('visual', gen_visualscores)
    gen_r = lambda shape: 2
    gen_v0 = lambda shape: np.random.randint(14, 18+1, shape)
    pop.set_parameter_generator('r', gen_r, 'population')
    pop.set_parameter_generator('v0', gen_v0, 'person')

    # Define and set symptom score function
    gen_symptomscores = lambda v,a,s0: np.maximum(a*v+s0, SMIN)
    pop.set_score_generator('symptom_noerror', gen_symptomscores)
    gen_a = lambda shape: slope_option
    gen_s0 = lambda shape: np.random.normal(6, 2, shape)
    pop.set_parameter_generator('a', gen_a, 'population')
    pop.set_parameter_generator('s0', gen_s0, 'person')

    # Define and set error functions
    #Multiplicative
    pop.set_score_generator('symptom', lambda s,C: s*C)
    gen_C_mul = lambda shape: np.random.uniform(1-error_option, 1+error_option, shape)
    pop.set_parameter_generator('C', gen_C_mul, 'day')

    # Repeat
    pops = PopulationList.full(npops, pop)
    
    # Return
    return pops

npersons=10
npops=10
poplists = [[get_poster_populations(slope, 0.3, npersons, npops) for slope in (1, 2, 3)], [get_poster_populations(slope, 0.5, npersons, npops) for slope in (1, 2, 3)]]
for i in range(len(poplists)):
    for j in range(len(poplists[i])):
        poplists[i][j].generate()

In [8]:
# Sampling

In [7]:
# Regressions

regresults = []
for i in range(len(poplists)):
    regresults.append([])
    for j in range(len(poplists[i])):
        regresults[i].append(poplists[i][j].regress_mixed(random_effect='intercept'))

C:\tools\Anaconda3\envs\smile\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\tools\Anaconda3\envs\smile\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\tools\Anaconda3\envs\smile\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\tools\Anaconda3\envs\smile\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2114: ConvergenceWarning: Retrying MixedLM optimization with cg
  ConvergenceWarning)
C:\tools\Anaconda3\envs\smile\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\

## Experiment 1: Baselines

Purpose:  
  * Evaluate the bias in an idealized, full-information setting. Evaluate the difference between different regression models in the absence of random effects in the population.  
  
Hypothesis:  
  * There will be no bias in noerror or multiplicative error. There will be a small bias towards a slower (lower) slope in additive error. Both regressions will give the same results. Both models will give identical results.  
  
Reasoning:  
  * The relationship is linear, no persons are discarded from the population, each person will have at least two points to describe their straight line. Multiplicative error will cancel out. However, for additive errors, the enduring error when the visual score is recovered will lead to more observations at (VMIN, +s) without equivalents at (VMIN, -s) to balance them out since the symptoms have a lower bound (see thick vertical line on the left of the graph of visual vs symptom).

In [5]:
# Simulation definition

def get_baseline_populations(npersons=100, npops=100):
    '''returns a PopulationList with no errors, a PopulationList with additive errors, and a PopulationList with multiplicative errors'''
    
    # Define and set visual score function
    pop_beforeerror = Population(size, 'linear')
    gen_visualscores = lambda t,r,v0: np.maximum(-r*t+v0, VMIN)
    pop_beforeerror.set_score_generator('visual', gen_visualscores)
    gen_r = lambda shape: truncatednormal(1/15, 31/15, 2, shape)
    gen_v0 = lambda shape: truncatednormal(14, 18, 2, shape)
    pop_beforeerror.set_parameter_generator('r', gen_r, 'person')
    pop_beforeerror.set_parameter_generator('v0', gen_v0, 'person')

    # Define and set symptom score function
    gen_symptomscores = lambda v,a: a*(v-VMIN)
    pop_beforeerror.set_score_generator('symptom_noerror', gen_symptomscores)
    gen_a = lambda shape: 1.5
    pop_beforeerror.set_parameter_generator('a', gen_a, 'population')

    # Define and set error functions
    #None
    pop_noerror = pop_beforeerror.copy(addtitle='no error')
    pop_noerror.set_score_generator('symptom', lambda s: s)
    #Multiplicative
    pop_mulerror = pop_beforeerror.copy(addtitle='multiplicative error')
    pop_mulerror.set_score_generator('symptom', lambda s,C: s*C)
    gen_C_mul = lambda shape: truncatednormal(0.5, 1.5, 2, shape)
    pop_mulerror.set_parameter_generator('C', gen_C_mul, 'day')
    #Additive
    pop_adderror = pop_beforeerror.copy(addtitle='additive error')
    pop_adderror.set_score_generator('symptom', lambda s,C: np.maximum(s+C, SMIN))
    gen_C_add = lambda shape: truncatednormal(-5, 5, 2, shape)
    pop_adderror.set_parameter_generator('C', gen_C_add, 'day')

    # Repeat
    pops_noerror = PopulationList.full(npops, pop_noerror, title='linear no error')
    pops_mulerror = PopulationList.full(npops, pop_mulerror, title='linear multiplicative error')
    pops_adderror = PopulationList.full(npops, pop_adderror, title='linear additive error')
    
    # Return
    return pops_noerror, pops_mulerror, pops_adderror

populationlists = get_baseline_populations()
for populationlist in populationlists:
    populationlist.generate()

In [ ]:
# Regressions

regressionresults_linear = []
regressionresults_mixed = []
for populationlist in populationlists:
    regressionresults_linear.append(populationlist.regress_linear())
    regressionresults_mixed.append(populationlist.regress_mixed())

In [ ]:
# Plotting

ncols = 2 #number of parameters, TODO
nrows = len(list_of_results)

fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=plt.figaspect(0.5*nrows/ncols)*2, sharey='col', squeeze=False)

for row in range(nrows):
    slope_truth = 1
    intercept_truth = slope_truth * -smile.global_params.VMIN
    list_of_results[row].plot_box(axes[row], ground_truths=[intercept_truth, slope_truth])
        
plt.tight_layout()
plt.show()
#dotted green line is the mean, solid green line is the median